## Polynomial Chaos Expansion Example 4

Author: Katiana Kontolati \
Date: December 8, 2020

In this example, PCE is used to generate a surrogate model for a given set of 3D data.

### Dette & Pepelyshev exponential function

$$ f(x) = 100(\exp{(-2/x_1^{1.75})} + \exp{(-2/x_2^{1.5})} + \exp{(-2/x_3^{1.25})})$$

**Description:**  Dimensions: 3

**Input Domain:**  This function is evaluated on the hypercube $x_i \in [0,1]$ for all i = 1,2,3.

**Reference:**  Dette, H., & Pepelyshev, A. (2010). Generalized Latin hypercube design for computer experiments. Technometrics, 52(4).

Import necessary libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from UQpy.Surrogates import *

Define the function.

In [2]:
def function(x):
    return 100*(np.exp(-2/(x[:,0]**1.75)) + np.exp(-2/(x[:,1]**1.5)) + np.exp(-2/(x[:,2]**1.25)))

Create a distribution object, generate samples and evaluate the function at the samples.

In [3]:
np.random.seed(1)

dist = Uniform(loc=0, scale=1)   

marg = [dist]*3
joint = JointInd(marginals=marg)

n_samples = 165
x = joint.rvs(n_samples)
y = function(x)  

Create an object from the PCE class.

In [4]:
max_degree = 5
polys = Polynomials(dist_object=joint, degree=max_degree) 

Compute PCE coefficients using least squares regression, Lasso regression and Ridge regression.

In [5]:
lstsq = PolyChaosLstsq(poly_object=polys)
pce = PCE(method=lstsq) 
pce.fit(x,y) 

lasso = PolyChaosLasso(poly_object=polys, learning_rate=0.01, iterations=1000, penalty=0.1)
pce2 = PCE(method=lasso) 
pce2.fit(x,y) 

ridge = PolyChaosRidge(poly_object=polys, learning_rate=0.01, iterations=1000, penalty=0.1)
pce3 = PCE(method=ridge) 
pce3.fit(x,y)

In [6]:
n_samples = 500
x_val = joint.rvs(n_samples)
y_val = function(x_val)

### Error Estimation

Validation error.

In [7]:
n_samples = 500
x_val = joint.rvs(n_samples)
y_val = function(x_val)

error = ErrorEstimation(surr_object=pce)
error2 = ErrorEstimation(surr_object=pce2)
error3 = ErrorEstimation(surr_object=pce3)
print('Error from least squares regression is: ', error.validation(x_val, y_val))
print('Error from LASSO is: ', error2.validation(x_val, y_val))
print('Error from Ridge is: ', error3.validation(x_val, y_val))

Error from least squares regression is:  9.25e-05
Error from LASSO is:  0.002566
Error from Ridge is:  0.0026148


### Moment Estimation

Returns mean and variance of the PCE surrogate.

In [8]:
n_mc = 1000000
x_mc = joint.rvs(n_mc)
y_mc = function(x_mc)    
mu = np.mean(y_mc)

print('Moments from least squares regression :', MomentEstimation(surr_object=pce).get())
print('Moments from LASSO regression :', MomentEstimation(surr_object=pce2).get())
print('Moments from Ridge regression :', MomentEstimation(surr_object=pce3).get())
print('Moments from MC integration: ', (round((1/n_mc)*np.sum(y_mc),6), round((1/n_mc)*np.sum((y_mc-mu)**2),6)))

Moments from least squares regression : (8.247977, 45.397944)
Moments from LASSO regression : (8.178689, 44.778469)
Moments from Ridge regression : (8.176812, 44.721672)
Moments from MC integration:  (8.25404, 45.383791)
